In [ ]:
import os
import numpy as np
import math
import pandas as pd

print('Make sure to confirm that the following path is the correct working directory')
print(os.getcwd())

In [ ]:
'''
Master Import Block (2-Degree Increments) [I of II]

Because the data is so large, we assume that at any point, only a single 
speed's worth of data is available (i.e. 180 files + the relevant 0-speed readings for a single, fixed Hz). 
This can be toggled by increasing the nVel parameter, but is NOT recommended (i.e., keep nVel = 1). 


PARAMETERS: 

startIdx -- int -- in {1, ..., 8}, represents the lowest Hz we are importing; Hz value is 5*startIdx. 
nVel     -- int -- in {1, ..., 8}, represents how many sequential Hz we are importing

nTrain   -- int -- number of training points at EACH velocity and angle to end up with
nTest    -- int -- number of testing points at EACH velocity and angle to end up with
N        -- int -- number of points to average over PER datum (effective sampling rate is nominal_rate/N)
numAng   -- int -- number of angle increments present in the data; should be a divisor of 360
numVel   -- int -- number of different Hz total
M        -- int -- number of rows of each file to sample (smaller = faster)

RESULT: 

Output (after manually cycling through numVel instances) is equal to numVel files of mags, angs, angsrad, 
and readings. These are combined in the next block. 
'''

raise ValueError('Do not run if you already have the csv data!')

raise ValueError('Make sure to confirm file path at bottom of this block!')

#
# BEGIN Parameters to alter by the user
#

dataPath = 'Data/'
trainPath = 'fooTrain/'
testPath = 'fooTest/'

# Designed to be iterable by simply changing startIdx with everything else fixed

startIdx = 1
nVel = 1

nTrain = 100
nTest = 20
N = 6
numAng = 180       # For 10-degree increments set this to 36 instead of 180
numVel = 8
M = 4000 
#
# END Parameters to alter by the user
#

#
# BEGIN derived parameters
# 
numTrain = nTrain*N    # End up with nTrain train points which represent averages over N consecutive samples
numTest = nTest*N      # End up with nTest test points which represent averages over N consecutive samples

kAng = int(360/numAng)
kVel = int(40/numVel)

RTL = M - numTrain - numTest - 500
RTH = RTL + numTest
RL = RTH + 400
RH = RL + numTrain

if RH > M:
    # Error checking. We want to min M (faster read-in, less wasteful), but also need to sample and 
    # leave some offset between train and test to prevent overfitting to temporal correlations
    raise ValueError('RH larger than max_rows (M)! Need to increase M or reduce {N, nTrain, nTest}')

# 
# END derived parameters 
# 

#
# BEGIN storage arrays to be written out at the end
# 
df = np.zeros((numAng*nVel*nTrain, 6))
dfT = np.zeros((numAng*nVel*nTest, 6))
mags = np.zeros(numAng*nVel*nTrain)
angs = np.zeros(numAng*nVel*nTrain)
magsT = np.zeros(numAng*nVel*nTest)
angsT = np.zeros(numAng*nVel*nTest)

tmpf = np.zeros((nTrain, 6))
tmpft = np.zeros((nTest, 6))
#
# END storage arrays to be written out at the end
# 

# Import the zero-speed data as to difference from the data with velocity
txtString2 = dataPath+'calibData_angle_0_samples_30000_frequency_1000_motor_0_Hz.txt'
df1 = np.loadtxt(txtString2, skiprows=1, max_rows=M, delimiter=',')

# Iterate over the set of velocities (startIdx to startIdx + nVel)
for hz in range(startIdx, startIdx+nVel):
    # Keep tabs on progress by printing hz value
    print(hz)
    print()
    # Iterate over the set of angles (numAng)
    # Assumed to be saved in even degree intervals (e.g. 10 degrees, 2 degrees, etc)
    for ang in range(numAng): 
        # Load the data and difference with zero-velocity readings
        txtString = dataPath+'calibData_angle_' + str(int(ang*kAng)) + '_samples_30000_frequency_1000_motor_'+str(int(hz*kVel))+'_Hz.txt'
        df0 = np.loadtxt(txtString, skiprows=1, max_rows=M, delimiter=',')
        
        # Pick out training and test data segments
        tmpf0 = (df0[RL:RH,:6] - df1[RL:RH,:6])
        tmpft0 = (df0[RTL:RTH,:6] - df1[RTL:RTH,:6])
        
        # Perform the filtering (averaging), condensing from {numTrain, numTest} to {nTrain, nTest}
        for k in range(nTrain):
            tmpf[k,:] = np.mean(tmpf0[N*k:N*(k+1),:], axis=0)
            if k < nTest:
                tmpft[k,:] = np.mean(tmpft0[N*k:N*(k+1),:], axis=0)
        
        # Put the (nTrain, 6), (nTest, 6) arrays into the larger df, dfT arrays in the right spot
        # This represents the train/test features (X)
        df[(numAng*nTrain*(hz-startIdx)+ang*nTrain):(numAng*nTrain*(hz-startIdx)+(ang+1)*nTrain),:6] = tmpf
        dfT[(numAng*nTest*(hz-startIdx)+ang*nTest):(numAng*nTest*(hz-startIdx)+(ang+1)*nTest),:6] = tmpft
        
        # Put the (ntrain, ) (nTest, ) arrays into the larger angs, mags, angsT, magsT in the right spot
        # This represents the train/test labels (y)
        angs[(numAng*nTrain*(hz-startIdx)+ang*nTrain):(numAng*nTrain*(hz-startIdx)+(ang+1)*nTrain)] = ang*kAng*np.ones(nTrain) # Angles in degrees
        mags[(numAng*nTrain*(hz-startIdx)+ang*nTrain):(numAng*nTrain*(hz-startIdx)+(ang+1)*nTrain)] = speeds[hz]*np.ones(nTrain)
        angsT[(numAng*nTest*(hz-startIdx)+ang*nTest):(numAng*nTest*(hz-startIdx)+(ang+1)*nTest)] = ang*kAng*np.ones(nTest) # Angles in degrees
        magsT[(numAng*nTest*(hz-startIdx)+ang*nTest):(numAng*nTest*(hz-startIdx)+(ang+1)*nTest)] = speeds[hz]*np.ones(nTest)
        


'''
Once we have finished, write the results to files
This allows us to index files by the idx of the run
Here, we assume nVel = 1
'''

np.savetxt(trainPath+'readings'+str(startIdx)+'.csv', df, delimiter=',')
np.savetxt(trainPath+'mags'+str(startIdx)+'.csv', mags, delimiter=',')
np.savetxt(trainPath+'angs'+str(startIdx)+'.csv', angs, delimiter=',')

np.savetxt(testPath+'readings'+str(startIdx)+'.csv', dfT, delimiter=',')
np.savetxt(testPath+'mags'+str(startIdx)+'.csv', magsT, delimiter=',')
np.savetxt(testPath+'angs'+str(startIdx)+'.csv', angsT, delimiter=',')

In [1]:
'''
Master Import Block (2-Degree Increments) [II of II]

Because the data is so large, we assume that at any point, only a single 
speed's worth of data is available (i.e. 180 files + the relevant 0-speed readings for a single, fixed Hz). 
This can be toggled by increasing the nVel parameter, but is NOT recommended (i.e., keep nVel = 1). 


INPUT: 

numVel files each of mags, angs, angsrad, and readings, named with iterated numbers from 1 to numVel. 

REQUIREMENTS: 

Parameters from Master Block I in the workspace, specifically: 

     nTrain
     nTest
     numAng
     numVel

OUTPUT: 

A single concatenated file for each of mags, angs, angsrad, and readings, which preserves proper labeling. 
'''

raise ValueError('Confirm that you actually need to run this function!')

raise ValueError('Check the path of the written files, below!')

trainPath = 'fooTrain/'
testPath = 'fooTest/'

df = np.zeros((numAng*numVel*nTrain, 6))
dfT = np.zeros((numAng*numVel*nTest, 6))
mags = np.zeros(numAng*numVel*nTrain)
magsT = np.zeros(numAng*numVel*nTest)
angs = np.zeros(numAng*numVel*nTrain)
angsT = np.zeros(numAng*numVel*nTest)

for k in range(1, 1+numVel):
    print(k)
    df[numAng*nTrain*(k-1):numAng*nTrain*k,:] = (pd.read_csv(trainPath+'readings'+str(k)+'.csv', header=None)).to_numpy()
    mags[numAng*nTrain*(k-1):numAng*nTrain*k] = (pd.read_csv(trainPath+'mags'+str(k)+'.csv', header=None)).to_numpy()[:,0]
    angs[numAng*nTrain*(k-1):numAng*nTrain*k] = (pd.read_csv(trainPath+'angs'+str(k)+'.csv', header=None)).to_numpy()[:,0]
    
    dfT[numAng*nTest*(k-1):numAng*nTest*k,:] = (pd.read_csv(testPath+'readings'+str(k)+'.csv', header=None)).to_numpy()
    magsT[numAng*nTest*(k-1):numAng*nTest*k] = (pd.read_csv(testPath+'mags'+str(k)+'.csv', header=None)).to_numpy()[:,0]
    angsT[numAng*nTest*(k-1):numAng*nTest*k] = (pd.read_csv(testPath+'angs'+str(k)+'.csv', header=None)).to_numpy()[:,0]
    


    
# Save the files in the desired format. 
# The old files loaded above can be kept or manually removed

np.savetxt(trainPath+'readings.csv', df, delimiter=',')
np.savetxt(trainPath+'mags.csv', mags, delimiter=',')
np.savetxt(trainPath+'angs.csv', angs, delimiter=',')
np.savetxt(trainPath+'angsrad.csv', angs*math.pi/180.0, delimiter=',')

np.savetxt(testPath+'readings.csv', dfT, delimiter=',')
np.savetxt(testPath+'mags.csv', magsT, delimiter=',')
np.savetxt(testPath+'angs.csv', angsT, delimiter=',')
np.savetxt(testPath+'angsrad.csv', angsT*math.pi/180.0, delimiter=',')

"\nMaster Import Block (2-Degree Increments) [II of II]\n\nBecause the data is so large, we assume that at any point, only a single \nspeed's worth of data is available (i.e. 180 files + the relevant 0-speed readings for a single, fixed Hz). \nThis can be toggled by increasing the nVel parameter, but is NOT recommended (i.e., keep nVel = 1). \n\n\nINPUT: \n\nnumVel files each of mags, angs, angsrad, and readings, named with iterated numbers from 1 to numVel. \n\nOUTPUT: \n\nA single concatenated file for each of mags, angs, angsrad, and readings, which preserves proper labeling. \n"